In [6]:
from sliderule import icesat2
import geopandas as gpd
from src.myio import read_gpkg, read_csv, write_parquet, read_dem, read_parquet
import pandas as pd

In [2]:

# 1. Ініціалізація клієнта
icesat2.init("slideruleearth.io", verbose=True)

In [5]:

# 2. Завантаження твого полігону
gdf = read_gpkg("basin_grid.gpkg")
print(gdf)



   id      left       top     right    bottom  row_index  col_index  fid_2  \
0   7  24.80705  48.03575  24.89705  47.94575          1          1      1   
1   8  24.80705  47.94575  24.89705  47.85575          2          1      1   
2  11  24.89705  48.12575  24.98705  48.03575          0          2      1   
3  12  24.89705  48.03575  24.98705  47.94575          1          2      1   
4  13  24.89705  47.94575  24.98705  47.85575          2          2      1   
5  14  24.89705  47.85575  24.98705  47.76575          3          2      1   
6  15  24.89705  47.76575  24.98705  47.67575          4          2      1   
7  18  24.98705  47.94575  25.07705  47.85575          2          3      1   
8  19  24.98705  47.85575  25.07705  47.76575          3          3      1   
9  20  24.98705  47.76575  25.07705  47.67575          4          3      1   

   cat  value label                                           geometry  
0    1      1        MULTIPOLYGON (((24.72059 47.94575, 24.72029 47.

In [ ]:



# 3. Порожній список для результатів
results = []

# 4. Цикл по кожному полігону
for idx, row in gdf.iterrows():
    print(f"🔄 Обробка полігону {idx+1}/{len(gdf)}")

    try:
        region = icesat2.toregion(row.geometry)

        # 5. Параметри SlideRule
        parms = {
            "poly": region["poly"],
            "cnf": 4,
            "ats": 20.0,
            "cnt": 10,
            "len": 40.0,
            "res": 20.0
        }

        # 6. Запит ATL06
        df = icesat2.atl06p(parms)

        if not df.empty:
            df["grid_id"] = idx  # додай ID полігону
            results.append(df)

    except Exception as e:
        print(f"⚠️ Помилка в полігоні {idx}: {e}")

# 7. Об’єднання всіх результатів
if results:
    combined = gpd.GeoDataFrame(pd.concat(results, ignore_index=True), crs="EPSG:4326")
    combined.to_file("icesat2_grid_data.geojson", driver="GeoJSON")
    print("✅ Дані збережено у icesat2_grid_data.geojson")
else:
    print("❌ Немає результатів")

